Import the required libraries:


In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import pandas as pd
import cudf
from cuml.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


Load your data into a pandas DataFrame

In [75]:
test_sequence = "AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER"
mb_raw_data = pd.read_csv('C:\\Users\\Simon\\PycharmProjects\\ion-mobility-prediction\\data\\mann_bruker.txt', sep='\t')
mb_data_frame = pd.DataFrame(mb_raw_data)
# Delete all columns except Sequence, m/z, and CCS
mb_clean_frame = mb_data_frame[['Sequence', 'm/z', 'CCS']]
# Log transform CCS values
mb_clean_frame['CCS'] = mb_clean_frame['CCS'].apply(lambda x: np.log(x))

C:\Users\Simon\AppData\Local\Temp\ipykernel_18972\3676896777.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mb_clean_frame['CCS'] = mb_clean_frame['CCS'].apply(lambda x: np.log(x))


In [31]:
# Load in mass triplets seperated by tab
mass = pd.read_csv('C:\\Users\\Simon\\PycharmProjects\\ion-mobility-prediction\\data\\mass.txt', sep='\t')
#Calculate mass of each sequence and add to new Mass column of mb_data_frame
for AA in mb_data_frame['Sequence']:
    aa_mass = 0
    for i in AA:
        aa_mass += mass['Mass'].loc[mass['Amino Acid'] == i].values[0]
    mb_data_frame['Mass'].loc[mb_data_frame['Sequence'] == AA] = aa_mass

C:\Users\Simon\AppData\Local\Temp\ipykernel_18972\1976600104.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mb_data_frame['Mass'].loc[mb_data_frame['Sequence'] == AA] = aa_mass


KeyboardInterrupt: 

In [ ]:
print(mb_data_frame['Mass'].loc[mb_data_frame['Sequence'] == 'AAAAAAAAAPAAAATAPTTAATTAATAAQ'])
print(mb_data_frame)

Preprocess the data:

In [74]:
X = mb_clean_frame[['m/z']]
# Log transform CCS values
y = mb_clean_frame['CCS'].values

In [ ]:
data = cudf.read_csv('C:\\Users\\Simon\\PycharmProjects\\ion-mobility-prediction\\data\\mann_bruker.txt', sep='\t')

# Split the data into input features (X) and output variable (y)
X = data[['m/z']]
# Log transform CCS values
y = data['CCS'].applymap(lambda x: np.log(x))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a random forest regressor
rf_model = RandomForestRegressor(n_estimators=100)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Convert the cuDF Series to a NumPy array for compatibility with scikit-learn metrics
y_test_np = y_test.to_array()

# Evaluate the model
mse = mean_squared_error(y_test_np, y_pred)
print('Mean Squared Error:', mse)

Create the Random Forest model:

In [55]:
model = RandomForestRegressor()

Perform k-fold cross-validation with K=5:

In [51]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kfold, scoring='r2')

KeyboardInterrupt: 

Train the model on the entire dataset (optional):

In [56]:
model.fit(X, y)
mean_score = scores.mean()
std_score = scores.std()

NameError: name 'scores' is not defined

In [27]:
print(f"Mean R2 score: {mean_score}")
print(f"Standard deviation of R2 scores: {std_score}")

Mean R2 score: 0.49302277450851467
Standard deviation of R2 scores: 0.0022737052318398086


In [69]:

# get the m/z of the sequence
value = mb_data_frame['m/z'].loc[mb_data_frame['Sequence'] == test_sequence].values[0]
new_data = pd.DataFrame({'m/z': [value]})
preds = model.predict(new_data)
print(preds)
# Get the CCS value of the sequence
ccs = mb_data_frame['CCS'].loc[mb_data_frame['Sequence'] == test_sequence].values[0]
print(np.log(ccs))

[-79.63264586]
-79.61439294495536
